In [1]:
import sys
from pandas import read_csv
from itertools import groupby
from operator import itemgetter
from collections import Iterable
import re

In [2]:
def printList(mylist):
    print"[",
    for item in mylist:
        if item != None:
            print item+",",
    print"]"

In [3]:
def flatten(mylist, outlist,ignore_types=(str, bytes)):
    if mylist !=[]:
        for item in mylist:
            if isinstance(item, Iterable) and not isinstance(item, ignore_types):
                flatten(item, outlist)
            else:
                item=item.strip(' \t\n\r')
                outlist.append(item)
    return outlist

In [4]:
def consecutive_cap(tweetWordList_cappos):
    output=[]
    #identifies consequtive numbers in the sequence
    for k, g in groupby(enumerate(tweetWordList_cappos), lambda (i, x): i-x):
        output.append(map(itemgetter(1), g))
    return output

In [64]:
def abbrv_algo(phrase):
    '''abbreviation algorithm 
    trailing apostrophe:
           |period:
           |     multiple letter-period sequence:
           |         all caps
           | non period:
           |     ?/! else drop apostrophe
    else:
        unchanged
    '''
    p=re.compile(r'[^a-zA-Z\d\s]$')
    match_list = p.findall(phrase)
    if len(match_list)>0:
        if phrase.endswith('.'):
            p1= re.compile(r'([a-zA-Z][\.]\s*)')
            match_list = p1.findall(phrase)
            if len(match_list)>1:
                print "Found abbreviation: "+phrase
            else:
                phrase= phrase[:-1]
        else:
            if not (phrase.endswith('?')|phrase.endswith('!')):
                phrase= phrase[:-1]
    else:
        p2=re.compile(r'[^a-zA-Z0-9_\s]')
        match_list = p.findall(phrase)
        if ((len(match_list)==0) & (phrase.isupper())):
            print "Found abbreviation: "+phrase
            
    return phrase
    

In [65]:
def punct_clause(cap_phrases):
    
    cap_phrases=cap_phrases.strip()
    
    if (re.compile(r'[^a-zA-Z0-9_\s]')).findall(cap_phrases):
        #case of intermediate punctuations: handles abbreviations
        p1= re.compile(r'(?:[a-zA-Z][^a-zA-Z0-9_\s]\s*)')
        match_lst = p1.findall(cap_phrases)
        

        index= (list( p1.finditer(cap_phrases) )[-1]).span()[1]
        p= re.compile(r'[^a-zA-Z\d\s]')
        match_list = p.findall(cap_phrases)

        p2=re.compile(r'[^a-zA-Z\d\s]$') #ends with punctuation

        if len(match_list)-len(match_lst)>0:
            if (p2.findall(cap_phrases)):
                #only strips trailing punctuations, not intermediate ones following letters
                cap_phrases = cap_phrases[0:index]+re.sub(p, '', cap_phrases[index:])
        
    
    #comma separated NEs
    lst=filter(lambda(word): word!="", re.split('[,]', cap_phrases))
    #print(lst)
    
    #check abbreviation
    final_lst= map(lambda phrase: abbrv_algo(phrase), lst)
    #print(lst)
    return final_lst

In [66]:
#%%timeit -o
def f(x,tweetWordList):
    '''phrase=""
    for word in x[:-1]:
        phrase+=tweetWordList[word]+" "
    phrase+=tweetWordList[x[-1]]'''
    
    list1=map(lambda word: tweetWordList[word]+" " , x[:-1])
    phrase="".join(list1)+tweetWordList[x[-1]]
    return phrase

In [67]:
df = read_csv('/home/satadisha/Desktop/GitProjects/ELTweetTracker/Collection.csv', index_col='ID', header=0, encoding='utf-8')
#print df.columns.values.tolist()

In [68]:
#%%timeit -o

count=0
for index, row in df.iterrows():
    
    if count<100:
        tweetText=unicode(row['Tweets']).encode('utf-8')
        #tweetText=row['Tweets']
        print(tweetText)
        #capitalization module
        #if all words are capitalized:
        if tweetText.isupper():
            print "All caps module"
        else:
            
            #returns word list split at whitespaces; retains punctuation
            tweetWordList=tweetText.split()
            '''print"[",
            for (i, j) in enumerate(tweetWordList):
                print j+", ",
            print"]"'''
            #returns position of words that are capitalized
            tweetWordList_cappos = map(lambda (index, word) : index, filter(lambda (index, word) : word[0].isupper(), enumerate(tweetWordList)))
            #print tweetWordList_cappos
            output= consecutive_cap(tweetWordList_cappos)
            
            #consecutive capitalized phrases
            consecutive_cap_phrases=map(lambda x: f(x,tweetWordList), output)
            #printList(consecutive_cap_phrases)
                        
            #implement the punctuation clause
            ne_List=flatten(map(lambda phrase: punct_clause(phrase), consecutive_cap_phrases),[])
            printList(ne_List)
            print ""
            
        count+=1
    else:
        break

U.S.  Presidential Election series Trump, Clinton,Obama wins big, Rubio...?
[ U.S. Presidential Election, Trump, Clinton, Obama, Rubio, ]

If Trump wins the presidential election, I'll make a logo for whoever RTs this tweet.
[ If Trump, I'll, RTs, ]

Insanely Accurate' Election Model Has Been Wrong Only Once Since 1912. Guess Who It Says is The Winner?
[ Insanely Accurate' Election Model Has Been Wrong Only Once Since, Guess Who It Says, The Winner?, ]

If you're eligible to vote in the US Presidential Election, make sure you hit up the polls TODAY 
Found abbreviation: TODAY
[ If, US Presidential Election, TODAY, ]

Trump’s “rigged election” claims aren’t new. Conservatives have waged war on electoral democracy since its inception
[ Trump’s, Conservatives, ]

Donald Trump got his Twitter confiscated and Obama destroyed him over it
[ Donald Trump, Twitter, Obama, ]

This is the 5th time I voted in a presidential election, but the 1st time I voted with my vagina

And now there's a pen st

In [58]:
st="Clinton"
print st.isupper()

False
